In [ ]:
# Import packages
# To ignore unimporant system warnings
import warnings
warnings.filterwarnings("ignore")

# We will use Pandas, Numpy, and Matplotlib which is a package for visualization with Python
import pandas as pd
import numpy as np

# This is a library for accessing and parsing data through URLs
from urllib.parse import urlencode
import urllib.request, json 
from bs4 import BeautifulSoup # for web scraping
import fiona
import plotly.express as px
import geopandas as gpd
import adjustText as aT
import seaborn as sns # visualization styling package
import plotly.graph_objects as go
import urllib, json
import os
import plotly
from sodapy import Socrata
from IPython.display import Image, display
from shapely.geometry import Polygon
import time
from shapely.geometry import shape
import contextily as ctx
#from google.colab import files
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline 
import networkx as nx #library supporting networks
import matplotlib.pyplot as plt
import seaborn as sns # visualization styling package
import folium
from folium.plugins import HeatMap


from dotenv import load_dotenv
import os
load_dotenv()
MAPS_API_KEY = os.getenv('MAPS_API_KEY')

In [ ]:
# import more packages
import plotly.io as pio
pio.renderers.default = "colab"
access_token = 'pk.eyJ1IjoiYmtpcnRoaSIsImEiOiJja3VpdzFiMnoycnYyMzBtYXM1aW1uemd0In0.knHw7eYrN8DvrrBHZugW8Q'
px.set_mapbox_access_token(access_token) # for interactive plotly viz

import plotly.graph_objects as go
import chart_studio.plotly as py
import chart_studio.tools as tls

In [ ]:
from shapely.geometry import Polygon, LineString, MultiLineString, Point, GeometryCollection
from pyproj import Geod
from shapely import wkt

#Load Data

In [ ]:
from urllib.parse import urlencode
import urllib.request, json 
import requests

In [ ]:
%cd ..
path = 'inputs/Data Axle Historic Businesses/tkabaix9xdeir06x.csv'
hdf = pd.read_csv(path, encoding = 'unicode_escape')

In [ ]:
bdf = hdf[['abi','archive_version_year','company','zipcode', 'primary_naics_code',
       'naics8_descriptions','year_established', 'latitude', 'longitude']]
bdf['primary_naics_code'].fillna(0,inplace=True)
bdf['primary_naics_code'] = bdf['primary_naics_code'].astype(int).astype(str)
bdf['naics'] = bdf['primary_naics_code'].str[:2]
bdf['naics'] = bdf['naics'].astype(int)
bdf = bdf[((bdf['naics'] >= 42) & (bdf['naics'] <= 54)) | ((bdf['naics'] == 72) | (bdf['naics'] == 71))]

bdf.archive_version_year = pd.to_datetime(bdf.archive_version_year, format='%Y')
bdf.archive_version_year = pd.DatetimeIndex ( bdf.archive_version_year ).astype ( np.int64 )/1000000
bdf.archive_version_year = bdf.archive_version_year.astype(int)

bdf['instance'] = bdf.groupby('abi').cumcount()+1
bdf.to_csv('outputs/Detroit_Biz.csv')



In [ ]:
path = 'inputs/ZCTA Boundaries/tl_2021_us_zcta520/tl_2021_us_zcta520.shp'
zcta = gpd.read_file(path)
zcta = zcta[['ZCTA5CE20','geometry']]
zcta['ZCTA5CE20'] = zcta['ZCTA5CE20'].astype(int)

In [ ]:
path = 'inputs/area_of_interest_zctas.csv'
dt = pd.read_csv(path)
dt.rename(columns={'ZCTA':'ZCTA5CE20', '"City" Name':'City Name'},inplace=True)

dt = dt.merge(zcta,on='ZCTA5CE20')
df = gpd.GeoDataFrame(dt, geometry='geometry')

In [ ]:
# import urllib library
from urllib.request import urlopen
import json

In [ ]:
def get_meta(lat, lon, api_key):
    url = "https://maps.googleapis.com/maps/api/streetview/metadata?location={0:},%20{1:}&key={2:}"
    response = json.loads(urlopen(url.format(lat, lon)).read())
    if response['status'] == 'OK':
        url = 'https://www.google.com/maps/@{0:},{1:}/data=!3m7!1e1!3m5!1s{2:}!2e0!5s20201001T000000!7i16384!8i8192'
        stview = url.format(lat, lon, response['pano_id'])
        return response['status'], response, stview
    else:
        return response['status']

In [ ]:
la = 41.762082
lo = -83.481032

In [ ]:
get_meta(la, lo, MAPS_API_KEY)   

In [ ]:
%run points.py

In [ ]:
la=42.5773904342342
lo=-83.1436338234234234
panoids(la,lo, MAPS_API_KEY)

In [ ]:
df = pd.DataFrame(panoids(la, lo, MAPS_API_KEY)).fillna(0)
df['lat_og'] = df['lat']
df['lon_og'] = df['lon']
df.loc[df['year'] != 0, 'lat_og'] = la
df.loc[df['year'] != 0, 'lon_og'] = lo

In [ ]:
#ndf1.to_csv('../outputs/Panoids/AllPoints.csv')

In [ ]:
ndf1 = pd.read_csv('../outputs/Panoids/AllPoints_NYC.csv')

In [ ]:
pdf = pd.DataFrame(columns=['panoid','lat','lon','year','month','lat_og','lon_og'])

Errors (None error) in lines 439, 566, 567, 573, 1225, 1819, 1822, 2511, 2796, 2927, 3387, 3358, 4222, 4628, 2631, 4632, 5692, ...

In [ ]:
KJPobXgZwSTXCtpDHgaI_Q
PIXnMHJ6Syg90cCxmjceaQ

In [ ]:
directory = './'
final_directory = './'
panoid = '9x8DXcEork_1arzq-5OkrA'
tiles = tiles_info(panoid)
download_tiles(tiles_info(panoid), directory)
stich_tiles(panoid, directory, final_directory)

In [ ]:
for i in range(103869,len(ndf1)):
    lo = ndf1.x[i]
    la = ndf1.y[i]
    if panoids(la,lo, MAPS_API_KEY) is not None:
        try:
            df = pd.DataFrame(panoids(la, lo, MAPS_API_KEY)).fillna(0)
            df['lat_og'] = df['lat']
            df['lon_og'] = df['lon']
            df.loc[df['year'] != 0, 'lat_og'] = la
            df.loc[df['year'] != 0, 'lon_og'] = lo
            pdf = pd.concat([pdf,df]).drop_duplicates().reset_index(drop=True)
        except (KeyError):
            print(i)
            continue

In [ ]:
pdf

In [ ]:
pdf.to_csv('../outputs/Panoids/AllPoints_NYC_1M.csv')

In [ ]:
panoids(la, lo, MAPS_API_KEY)

In [ ]:
lo = ndf1.x[439]
la = ndf1.y[439]
df = pd.DataFrame(panoids(la, lo, MAPS_API_KEY)).fillna(0)

In [ ]:
df

In [ ]:
pdf[(pdf.year != 0)][['lat_og','lon_og']].value_counts()

In [ ]:
#pdf.to_csv('../outputs/Panoids/0-120k.csv')

In [ ]:
#168
#H Mart

In [ ]:
panoid = df.panoid[6]

In [ ]:
directory = '/Volumes/GoogleDrive/My Drive/Columbia/3 Fall 22/RA/Detroit/Outputs'
final_directory = '/Volumes/GoogleDrive/My Drive/Columbia/3 Fall 22/RA/Detroit/Outputs'

In [ ]:
stich_tiles(panoid, directory, final_directory)

In [ ]:
directory = '/Users/kirthi/Library/CloudStorage/GoogleDrive-kb3185@columbia.edu/My Drive/Columbia/3 Fall 22/RA/Detroit/outputs/Panos'
final_directory = '/Users/kirthi/Library/CloudStorage/GoogleDrive-kb3185@columbia.edu/My Drive/Columbia/3 Fall 22/RA/Detroit/outputs/Panos'

In [ ]:
panoid = df.panoid[6]

In [ ]:
len(df)

In [ ]:
tiles = tiles_info(panoid)


In [ ]:
download_tiles(tiles_info(panoid), directory)
#stich_tiles(panoid, directory, final_directory)

In [ ]:
stich_tiles(panoid, directory, final_directory)

In [ ]:
import re

In [ ]:
#%run login.py

In [ ]:
URL = 'https://www.google.com/maps/@42.5472777,-83.1843046,3a,15y,196.83h,89.6t/data=!3m7!1e1!3m5!1sE71c0CE94Yv-635ySrLcbQ!2e0!5s20201001T000000!7i16384!8i8192'

In [ ]:
b = driver.driver

In [ ]:
b.get(URL)

In [ ]:
ele = b.find_elements(By.CLASS_NAME, "aLPB6c.kaqDpe")

In [ ]:
len(ele)

In [ ]:
pxdf = pd.DataFrame(columns=['panoid','lat','lon','year','month'])
zdf = pd.DataFrame(columns=['panoid','lat','lon'])
            
for i in range(130):
    x = i*10000
    print(x)
    path = 'outputs/Panoids/AllPanoids - ' +str(x)+ '.csv'
    df = pd.read_csv(path, error_bad_lines=False)
    temp_pdf = df[df['year'] != 0][['panoid','lat','lon','year','month']].reset_index(drop=True)
    pxdf = pd.concat([pxdf,temp_pdf]).drop_duplicates().reset_index(drop=True)
    temp_zdf = df[df['year'] == 0][['panoid','lat','lon']].reset_index(drop=True)
    zdf = pd.concat([zdf,temp_zdf]).drop_duplicates().reset_index(drop=True)

pxdf.to_csv('../outputs/Panoids/Valid - All.csv')
zdf.to_csv('../outputs/Panoids/Zero - All.csv')

In [ ]:
zdf

In [ ]:
pdf.to_csv('../outputs/Panoids/Valid.csv')
zdf.to_csv('../outputs/Panoids/Zero.csv')

In [ ]:
s = ele[1].get_attribute('outerHTML')

In [ ]:
s

In [ ]:
from selenium import webdriver
import undetected_chromedriver as uc
from time import sleep

driver_path = '/usr/local/bin/chromedriver.exe'

chromeOptions = uc.ChromeOptions() 
#chromeOptions = webdriver.ChromeOptions() 
chromeOptions.binary_location = '/Applications/Google Chrome Canary.app/Contents/MacOS/Google Chrome Canary'
#chromeOptions.executable_path = driver_path
#chromeOptions.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 1}) 
chromeOptions.add_argument("user-data-dir=/Users/kirthi/Library/Application Support/Google/Chrome Canary")
chromeOptions.add_argument("--no-sandbox") 
chromeOptions.add_argument("--disable-setuid-sandbox") 

chromeOptions.add_argument('--ignore-ssl-errors=yes')
chromeOptions.add_argument('--ignore-certificate-errors')
chromeOptions.add_experimental_option("useAutomationExtension", False)
chromeOptions.add_experimental_option("excludeSwitches",["enable-automation"])


#chromeOptions.add_argument("--remote-debugging-port=9515")  # this

#chromeOptions.add_argument("--disable-dev-shm-using") 
chromeOptions.add_argument("--disable-extensions") 
chromeOptions.add_argument("--disable-gpu") 
chromeOptions.add_argument("disable-infobars")

In [ ]:
import re

In [ ]:
[re.search(r"panoid=(.*?)\&", s).group(1), re.search(r"aria-label=\"(.*?)\"", s).group(1)]

In [ ]:
re.search(r"panoid=(.*?)\&", s).group(1),

In [ ]:
re.search(r"aria-label=\"(.*?)\"", s).group(1)

In [ ]:
b.quit()

In [ ]:
print('yuih')

In [ ]:
path = 'outputs/Panoids/Valid - All.csv'
dt = pd.read_csv(path)
dt.head()

In [ ]:
dt['lat'] = pd.to_numeric(dt['lat'], errors='coerce')

In [ ]:
dt['lon'] = pd.to_numeric(dt['lon'], errors='coerce')

In [ ]:
def filter_coordinates(df, min_lat, max_lat, min_long, max_long):
    return df[(df['lat'] >= min_lat) & (df['lat'] <= max_lat) & 
              (df['lon'] >= min_long) & (df['lon'] <= max_long)]

In [ ]:
bbox_df = filter_coordinates(dt, 42.560583, 42.567492, -83.093336, 83.081950)

In [ ]:
dt.shape

In [ ]:
bbox_df

In [ ]:
bbox_df = bbox_df.drop(columns={'Unnamed: 0'}).reset_index(drop=True)

In [ ]:
url = 'https://streetviewpixels-pa.googleapis.com/v1/tile?cb_client=maps_sv.tactile&panoid={}&x=0&y=0&zoom=0&nbt=1&fover=2'

In [ ]:
bbox_df1 = bbox_df.iloc[4840:]

In [ ]:
image_folder = 'outputs/Sample'
for i, row in bbox_df1.iterrows():
    image_url = url.format(row['panoid'])
    response = requests.get(image_url)
    image_name = f"{row['panoid']}.jpeg"  # Change the extension as per the image type
    image_path = os.path.join(image_folder, image_name)
    with open(image_path, 'wb') as f:
        f.write(response.content)

In [ ]:
import pandas as pd
import os
import re

folder_path = 'outputs/Cleaned Sample/'
files = os.listdir(folder_path)

# Remove ".jpeg" extension from each element
ids_from_files = [filename[:-5] for filename in files]

subset_df = bbox_df[bbox_df["panoid"].isin(ids_from_files)]

print(subset_df)


In [ ]:
directory = 'outputs/Cleaned Sample/Panos'
final_directory = 'outputs/Cleaned Sample/Panos'

In [ ]:
for item in ids_from_files:
    print (item)
    stich_tiles(item, directory, final_directory)

In [ ]:
def crop_images(input_folder, output_folder):
    # Create the output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Get a list of all files in the input folder
    files = os.listdir(input_folder)

    for file in files:
        # Construct the full path of the input file
        input_path = os.path.join(input_folder, file)

        # Open the image using Pillow
        image = Image.open(input_path)

        # Get the width and height of the image
        width, height = image.size

        # Calculate the new height for cropping
        new_height = height // 2

        # Calculate the top and bottom positions for cropping
        top = (height - new_height) // 2
        bottom = top + new_height

        # Crop the image
        cropped_image = image.crop((0, top, width, bottom))

        # Construct the full path of the output file
        output_path = os.path.join(output_folder, file)

        # Save the cropped image
        cropped_image.save(output_path)

        print(f"Cropped and saved {file}")

# Specify the input and output folders
input_folder = 'outputs/Cleaned Sample/Panos'
output_folder = 'outputs/Cleaned Sample/Cropped Panos'

# Call the function to crop and save the images
crop_images(input_folder, output_folder)